In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
import os, shutil,sys
import random

# Whithout augmentation using all level of fake images 

In [4]:
original_dataset_dir = '/home/yanru/summer2020/real_and_fake_face_orig'
training_r = '/home/yanru/summer2020/real_and_fake_face_orig/training_real'
training_f = '/home/yanru/summer2020/real_and_fake_face_orig/training_fake'
test_dir  = '/home/yanru/summer2020/real_and_fake_all/test'

train_dir = '/home/yanru/summer2020/real_and_fake_all/train'
validation_dir = '/home/yanru/summer2020/real_and_fake_all/validation'
train_real = '/home/yanru/summer2020/real_and_fake_all/train/training_real'
train_fake = '/home/yanru/summer2020/real_and_fake_all/train/training_fake'
real_validation = '/home/yanru/summer2020/real_and_fake_all/validation/validation_real'
fake_validation = '/home/yanru/summer2020/real_and_fake_all/validation/validation_fake'
test_real = '/home/yanru/summer2020/real_and_fake_all/test/testing_real'
test_fake = '/home/yanru/summer2020/real_and_fake_all/test/testing_fake'


# make a list of real imges 
list_real = os.listdir(training_r)
seed=42
random_list_real = random.sample(list_real,len(list_real))
#copy images to training real
fnames = [random_list_real[i] for i in range (0,500)]
for fname in fnames:
    src = os.path.join(training_r, fname)
    dst = os.path.join(train_real, fname)
    shuticl.copyfile(src, dst)

print("training real:", len(os.listdir(train_real)))

# copy images to validation real
fnames = [random_list_real[i] for i in range (500,800)]
for fname in fnames:
    src = os.path.join(training_r, fname)
    dst = os.path.join(real_validation, fname)
    shutil.copyfile(src, dst)
    
print("validation real:", len(os.listdir(real_validation)))


# copy images to test real
fnames = [random_list_real[i] for i in range (800,1081)]
for fname in fnames:
    src = os.path.join(training_r, fname)
    dst = os.path.join(test_real, fname)
    shutil.copyfile(src, dst)
print("test real:", len(os.listdir(test_real)))    

# create a list of fake images 
list_fake = os.listdir(training_f)
#make is random
seed=42
random_list_fake = random.sample(list_fake,len(list_fake))


print(random_list_fake[0])

nfnames = [random_list_fake[i] for i in range (0,400)]
for fname in fnames:
    src = os.path.join(training_f, fname)
    dst = os.path.join(train_fake, fname)
    shutil.copyfile(src, dst)

print("training fake:", len(os.listdir(train_fake)))

fnames = [random_list_fake[i] for i in range (400,700)]
for fname in fnames:
    src = os.path.join(training_f, fname)
    dst = os.path.join(fake_validation, fname)
    shutil.copyfile(src, dst)
print("validation fake:", len(os.listdir(fake_validation)))


fnames = [random_list_fake[i] for i in range (700,960)]
for fname in fnames:
    src = os.path.join(training_f, fname)
    dst = os.path.join(test_fake, fname)
    shutil.copyfile(src, dst)
print("test fake:", len(os.listdir(test_fake)))







In [6]:
from keras import layers
from keras import models

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))




In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

In [9]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
                    train_dir,target_size = (150,150),batch_size = 20,class_mode = 'binary')



FileNotFoundError: [Errno 2] No such file or directory: '/home/yanru/summer2020/real_and_fake_face_small/train'

In [ ]:
validation_generator = test_datagen.flow_from_directory(
                validation_dir,target_size = (150,150),batch_size = 20,class_mode = 'binary')

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('fake_real_images_all2')

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

# With augmentation

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('fake_real_images_small_aug')

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)